In [25]:
import sxsc_tushare as sx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import *
from numpy.linalg import multi_dot
sx.set_token("0a3191104cd108eccc6d1d4ce8a431f8d99fb786926e5d788f3b55c5")#因为这个账号是私有的 所以老师我无法把真实通行token给到你
pro = sx.get_api(env="prd")

In [27]:
df = pro.daily( ts_code = ' 600298.SH, 600872.SH, 600186.SH', start_date='20041231', end_date='20241231', )
df.head(10)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,600872.SH,20241231,22.90,22.95,21.97,22.02,22.88,-0.86,-3.7587,128218.61,286369.862
1,600298.SH,20241231,36.26,36.57,36.05,36.05,36.26,-0.21,-0.5792,46909.80,170140.988
2,600186.SH,20241231,5.02,5.70,4.92,5.70,5.18,0.52,10.0386,2546273.82,1386903.837
3,600186.SH,20241230,5.39,5.39,5.11,5.18,4.90,0.28,5.7143,1863664.92,980407.231
4,600298.SH,20241230,36.32,36.71,36.10,36.26,36.37,-0.11,-0.3024,41062.20,149470.040
5,600872.SH,20241230,23.27,23.36,22.75,22.88,23.26,-0.38,-1.6337,113979.32,261826.508
6,600872.SH,20241227,23.31,23.43,22.98,23.26,23.27,-0.01,-0.0430,87356.00,202761.922
7,600298.SH,20241227,35.90,36.55,35.83,36.37,35.97,0.40,1.1120,43404.66,157068.130
8,600186.SH,20241227,4.83,5.01,4.75,4.90,4.78,0.12,2.5105,529206.00,259854.243
9,600186.SH,20241226,4.70,4.84,4.67,4.78,4.74,0.04,0.8439,366488.89,175079.632


In [28]:
df['trade_date'] = pd.to_datetime(df['trade_date'], format='%Y%m%d')
# df = df.sort_values('trade_date' )ascending = True
df = df.sort_values('trade_date', ascending=True)

In [29]:
df_filtered = df[['ts_code', 'trade_date', 'close']]

In [33]:
df_filtered

,ts_code,trade_date,close
5999,600186.SH,2016-10-11,5.50
5996,600298.SH,2016-10-12,16.92
5998,600186.SH,2016-10-12,5.46
5997,600872.SH,2016-10-12,14.99
5993,600186.SH,2016-10-13,5.53
...,...,...,...
4,600298.SH,2024-12-30,36.26
3,600186.SH,2024-12-30,5.18
2,600186.SH,2024-12-31,5.70
1,600298.SH,2024-12-31,36.05


In [35]:
df_pivot = df_filtered.pivot(
    index='trade_date',   # 行索引为日期
    columns='ts_code',    # 列索引为股票代码
    values='close'        # 填充值为收盘价
)



In [37]:
df_pivot 

ts_code,600186.SH,600298.SH,600872.SH
trade_date,,,
2016-10-11,5.50,NaN,NaN
2016-10-12,5.46,16.92,14.99
2016-10-13,5.53,17.05,14.98
2016-10-14,5.52,16.96,15.69
2016-10-17,5.41,17.30,15.44
...,...,...,...
2024-12-25,4.74,35.44,23.12
2024-12-26,4.78,35.97,23.27
2024-12-27,4.90,36.37,23.26


In [39]:
df_filled = df_pivot.fillna(method='bfill', axis=0)
df_filled


C:\Users\perco\AppData\Local\Temp\ipykernel_17752\3706142441.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled = df_pivot.fillna(method='bfill', axis=0)


ts_code,600186.SH,600298.SH,600872.SH
trade_date,,,
2016-10-11,5.50,16.92,14.99
2016-10-12,5.46,16.92,14.99
2016-10-13,5.53,17.05,14.98
2016-10-14,5.52,16.96,15.69
2016-10-17,5.41,17.30,15.44
...,...,...,...
2024-12-25,4.74,35.44,23.12
2024-12-26,4.78,35.97,23.27
2024-12-27,4.90,36.37,23.26


In [41]:
returns = (df_filled/df_filled.shift(1)).dropna()

returns.shape

(2001, 3)

In [57]:
# 组合的股票构成

symbols = ['600186.SH','600298.SH','600872.SH']
numofasset = len(symbols)
 
# 定义一个组合的函数
def portfolio_simulation(returns):
    rets = []; vols=[]; wts=[]
    
    # 模拟10000次3只股票不同权重的组合
    for i in tqdm(range(10000)):
        # 产生随机权重
        weights = np.random.random(numofasset)[:, np.newaxis]
        weights = weights/weights.sum()
        
        # 记录组合的收益、波动、权重
        rets.append(weights.T @ np.array(returns.mean()*250)[:,np.newaxis])
        vols.append(np.sqrt(multi_dot([weights.T, returns.cov()*250, weights])))
        wts.append(weights.flatten())
        
    # 记录组合各数据
    portdf =  pd.DataFrame({
        'port_rets': np.array(rets).flatten(),
        'port_vols': np.array(vols).flatten(),
        'weights': list(np.array(wts))
        })
    portdf['Sharp_ratio']=(portdf['port_rets']-2)/(portdf['port_vols']*100)  #假设rf为0
    
    return round(portdf, 2)

temp = portfolio_simulation(returns)
temp.head(10)

100%|██████████| 10000/10000 [00:12<00:00, 814.54it/s]


,port_rets,port_vols,weights,Sharp_ratio
0,250.13,0.31,"[0.1284964146949947, 0.29597011953521746, 0.57...",7.93
1,250.13,0.28,"[0.31328099522438213, 0.3722436975284381, 0.31...",8.74
2,250.13,0.29,"[0.35159200015001035, 0.5504615608485719, 0.09...",8.62
3,250.11,0.34,"[0.7234857476918675, 0.1295929090211507, 0.146...",7.27
4,250.13,0.28,"[0.34006573205216617, 0.3355235224405914, 0.32...",8.71
5,250.13,0.29,"[0.31393509532575065, 0.26080670916701365, 0.4...",8.52
6,250.13,0.29,"[0.33475246880383275, 0.5842450711956731, 0.08...",8.57
7,250.11,0.31,"[0.5135315374359992, 0.0877025145647942, 0.398...",7.98
8,250.13,0.29,"[0.2645984930482668, 0.35114177260255824, 0.38...",8.62
9,250.12,0.29,"[0.47067376945522316, 0.3966342449425514, 0.13...",8.51
